In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()
TRAIN_SET = os.getenv("TRAIN_PATH")
TEST_SET  = os.getenv("TEST_PATH")

train = pd.read_csv(TRAIN_SET)
test = pd.read_csv(TEST_SET) 

train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Estraggo la target feature: _'sii'_ 

In [ ]:
#extact the target variable: sii feature
y_train = train["sii"]
print(y_train, "\n", "=============================================")
print(y_train.shape, "\n", "=============================================")
print(y_train.info())

0       2.0
1       0.0
2       0.0
3       1.0
4       NaN
       ... 
3955    1.0
3956    NaN
3957    1.0
3958    0.0
3959    NaN
Name: sii, Length: 3960, dtype: float64 
(3960,) 
<class 'pandas.core.series.Series'>
RangeIndex: 3960 entries, 0 to 3959
Series name: sii
Non-Null Count  Dtype  
--------------  -----  
2736 non-null   float64
dtypes: float64(1)
memory usage: 31.1 KB
None


Notare che y_target ha il 75% circa di valori non-null (2736/3660) * 100

**Pulizia dei dati:**
 - Rimuovo le feature non presenti nel test set e stampo percentuali dei valori presenti
 

In [4]:
def intersect_features(tr, ts):
    smaller_train = tr[tr.columns.intersection(ts.columns)]

    # Crea una lista per memorizzare le righe
    rows = []

    # Per ogni feature nelle colonne comuni tra train e test
    for feature in smaller_train.columns:
        # Calcola la percentuale di valori non nulli nel train e nel test
        train_not_null_perc = 100 * smaller_train[feature].notnull().sum() / len(smaller_train)
        test_not_null_perc = 100 * test[feature].notnull().sum() / len(test)
        
        # Aggiungi il risultato alla lista
        rows.append({'feature': feature, 'train': train_not_null_perc, 'test': test_not_null_perc})

    # Converti la lista in un DataFrame
    features = pd.DataFrame(rows)
    features.sort_values('train', ascending=False, inplace=True)

    return smaller_train, features

intersect_features(train, test)
real_train = intersect_features(train, test)[0]
real_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3960 non-null   object 
 1   Basic_Demos-Enroll_Season               3960 non-null   object 
 2   Basic_Demos-Age                         3960 non-null   int64  
 3   Basic_Demos-Sex                         3960 non-null   int64  
 4   CGAS-Season                             2555 non-null   object 
 5   CGAS-CGAS_Score                         2421 non-null   float64
 6   Physical-Season                         3310 non-null   object 
 7   Physical-BMI                            3022 non-null   float64
 8   Physical-Height                         3027 non-null   float64
 9   Physical-Weight                         3076 non-null   float64
 10  Physical-Waist_Circumference            898 non-null    floa

 - Rimuovo le feature con una certa percentuale di valori NaN. Droppo anche la feature relativa all'id. Ho già droppato la target column

In [5]:
real_train_no_id = real_train.drop(columns=["id"], axis=1)

In [6]:
# Drop the columns with more than a fixed percentage of missing values:
def drop_columns(df, threshold = 0.3): #by default the threshold is 20%. If a column is >= 80% of NaNs, it will be dropped. It can be changed.

    # Calculate the minimum required non-NaN values per column based on the threshold percentage
    minimum_non_NaN = len(df) * threshold   

    # Identify columns to be dropped
    dropped_columns = df.columns[df.isnull().sum() > (len(df) - minimum_non_NaN)].tolist()

    # Drop the identified columns from the DataFrame
    new_df = df.drop(columns=dropped_columns)

    return new_df, dropped_columns

cleaned_train, dropped_columns = drop_columns(real_train_no_id)
cleaned_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 48 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Basic_Demos-Enroll_Season               3960 non-null   object 
 1   Basic_Demos-Age                         3960 non-null   int64  
 2   Basic_Demos-Sex                         3960 non-null   int64  
 3   CGAS-Season                             2555 non-null   object 
 4   CGAS-CGAS_Score                         2421 non-null   float64
 5   Physical-Season                         3310 non-null   object 
 6   Physical-BMI                            3022 non-null   float64
 7   Physical-Height                         3027 non-null   float64
 8   Physical-Weight                         3076 non-null   float64
 9   Physical-Diastolic_BP                   2954 non-null   float64
 10  Physical-HeartRate                      2967 non-null   floa

In [7]:
cleaned_train.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Diastolic_BP,...,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,...,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,75.0,...,39.4497,15.4107,27.0552,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,65.0,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,60.0,...,58.9338,26.4798,45.9966,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 - Divido le feature nel mio dataset tra numerical features e cathegorical features. Converto poi le cathegorical tramite 1HE

In [8]:
def extract_numerical_cathegorical(df):
    # Extract numerical and categorical columns
    numerical = df.select_dtypes(include=np.number).columns.tolist()
    categorical = df.select_dtypes(exclude=np.number).columns.tolist()

    return numerical, categorical

numerical_features, categorical_features = extract_numerical_cathegorical(cleaned_train)
print("Numerical features are", len(numerical_features), "; -> ", numerical_features)
print("Cathegorical features are: ", len(categorical_features), "; -> ", categorical_features)

Numerical features are 39 ; ->  ['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday']
Cathegorical features are:  9 ; ->  ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']


In [9]:
cleaned_train_1he = pd.get_dummies(cleaned_train, columns=categorical_features)
cleaned_train_1he.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 75 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Basic_Demos-Age                         3960 non-null   int64  
 1   Basic_Demos-Sex                         3960 non-null   int64  
 2   CGAS-CGAS_Score                         2421 non-null   float64
 3   Physical-BMI                            3022 non-null   float64
 4   Physical-Height                         3027 non-null   float64
 5   Physical-Weight                         3076 non-null   float64
 6   Physical-Diastolic_BP                   2954 non-null   float64
 7   Physical-HeartRate                      2967 non-null   float64
 8   Physical-Systolic_BP                    2954 non-null   float64
 9   FGC-FGC_CU                              2322 non-null   float64
 10  FGC-FGC_CU_Zone                         2282 non-null   floa

In [10]:
cleaned_train_1he.head()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,...,PAQ_C-Season_Summer,PAQ_C-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
0,5,0,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,0.0,...,False,False,False,False,False,False,True,False,False,False
1,9,0,NaN,14.035590,48.0,46.0,75.0,70.0,122.0,3.0,...,False,False,True,False,False,False,False,False,True,False
2,10,1,71.0,16.648696,56.5,75.6,65.0,94.0,117.0,20.0,...,True,False,True,False,False,False,False,False,True,False
3,9,0,71.0,18.292347,56.0,81.6,60.0,97.0,117.0,18.0,...,False,True,False,False,True,False,False,False,False,True
4,18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


Ci sono 39 features numeriche e 36 features categoriche codificate tramite 1HE. Le 1HE sono sottoforma di true/false. Le rendo features numeriche:
 - false -> 0
 - true  -> 1

Così da mantenerle colonne sparse.
Successivamente converto tutte le colonne con lo stesso tipo numerico in _float64_

In [12]:
cleaned_train_1he *= 1
cleaned_train_1he = cleaned_train_1he.astype('float64')
cleaned_train_1he.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 75 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Basic_Demos-Age                         3960 non-null   float64
 1   Basic_Demos-Sex                         3960 non-null   float64
 2   CGAS-CGAS_Score                         2421 non-null   float64
 3   Physical-BMI                            3022 non-null   float64
 4   Physical-Height                         3027 non-null   float64
 5   Physical-Weight                         3076 non-null   float64
 6   Physical-Diastolic_BP                   2954 non-null   float64
 7   Physical-HeartRate                      2967 non-null   float64
 8   Physical-Systolic_BP                    2954 non-null   float64
 9   FGC-FGC_CU                              2322 non-null   float64
 10  FGC-FGC_CU_Zone                         2282 non-null   floa

 - Tramite bootstrap sampling sostituisco i valori NaN con valori stimati

In [ ]:
cleaned_train_1he = cleaned_train_1he.fillna(cleaned_train_1he.mean())
cleaned_train_1he.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 75 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Basic_Demos-Age                         3960 non-null   float64
 1   Basic_Demos-Sex                         3960 non-null   float64
 2   CGAS-CGAS_Score                         3960 non-null   float64
 3   Physical-BMI                            3960 non-null   float64
 4   Physical-Height                         3960 non-null   float64
 5   Physical-Weight                         3960 non-null   float64
 6   Physical-Diastolic_BP                   3960 non-null   float64
 7   Physical-HeartRate                      3960 non-null   float64
 8   Physical-Systolic_BP                    3960 non-null   float64
 9   FGC-FGC_CU                              3960 non-null   float64
 10  FGC-FGC_CU_Zone                         3960 non-null   floa

Equivalentemente stimo i valori per la variabile target:

In [18]:
y_train = y_train.fillna(y_train.mean())
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 3960 entries, 0 to 3959
Series name: sii
Non-Null Count  Dtype  
--------------  -----  
3960 non-null   float64
dtypes: float64(1)
memory usage: 31.1 KB


**Model training:**

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score  

model = RandomForestRegressor(n_estimators=100)
model.fit(cleaned_train_1he, y_train)

RandomForestRegressor()